In [6]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn import model_selection, naive_bayes, svm

from collections import Counter
from itertools import chain
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from html.parser import HTMLParser
import unicodedata
wnl = WordNetLemmatizer()
html_parser = HTMLParser()
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.decomposition import NMF
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, auc, roc_auc_score
from scipy.sparse import hstack
from scipy.sparse.linalg import svds
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import nltk
from gensim import corpora, models
# reference: https://github.com/dipanjanS/text-analytics-with-python/blob/master/Old-First-Edition/source_code/Ch04_Text_Classification/normalization.py
from pattern.en import tag
from nltk.corpus import wordnet as wn
import string
from nltk.stem import WordNetLemmatizer

import re
import random

stopword_list = nltk.corpus.stopwords.words('english')
stopword_list = stopword_list + ['mr', 'mrs', 'come', 'go', 'get', 'tell',
'listen', 'one', 'two', 'three', 'four', 'five',
'six', 'seven', 'eight',
'nine', 'zero', 'join', 'find', 'make', 'say', 'ask',
'tell', 'see', 'try', 'back', 'also']

In [13]:
df = pd.read_csv("mydata.csv")
df = df.drop(["Unnamed: 0"],axis = 1)

In [14]:
df.head()

Speaker                                           FileName  \
0    BUSH  ../Project/Data/BUSH/100th Anniversary of Cuba...   
1    BUSH  ../Project/Data/BUSH/2004 Presidential Electio...   
2    BUSH  ../Project/Data/BUSH/2004 Republican National ...   
3    BUSH  ../Project/Data/BUSH/2008 Republican National ...   
4    BUSH  ../Project/Data/BUSH/911 Address to the Nation...   

                                             RawText  \
0   thank you very much. sientese. voy a hablar e...   
1   thank you all. thank you all for coming. we h...   
2   mr. chairman! mr. chairman, delegates, fellow...   
3   thank you, laura, and good evening. as you kn...   
4   good evening.  today, our fellow citizens, ou...   

                                               Token  \
0  ['thank', 'much', 'sientese', 'voy', 'hablar',...   
1  ['thank', 'thank', 'coming', 'long', 'night', ...   
2  ['chairman', 'chairman', 'delegates', 'fellow'...   
3  ['thank', 'laura', 'good', 'evening', 'know', ...   
4  ['good', 'evening', 'today', 'fellow', 'citize...   

                                        Cleaned_text  \
0  thank much sientese voy hablar en espanol hoy ...   
1  thank thank coming long night great night vote...   
2  chairman chairman delegates fellow citizens ho...   
3  thank laura good evening know duties washingto...   
4  good evening today fellow citizens way life fr...   

                                          Sent_Token  \
0  [' thank you very much.', 'sientese.', 'voy a ...   
1  [' thank you all.', 'thank you all for coming....   
2  [' mr. chairman!', 'mr. chairman, delegates, f...   
3  [' thank you, laura, and good evening.', 'as y...   
4  [' good evening.', 'today, our fellow citizens...   

                                     Normalized_list  
0  [['thank', 'much'], ['sientese'], ['voy', 'hab...  
1  [['thank'], ['thank'], ['long', 'night', 'grea...  
2  [['chairman'], ['chairman', 'delegate', 'fello...  
3  [['thank', 'laura', 'good', 'evening'], ['know...  
4  [['good', 'evening'], ['today', 'fellow', 'cit...

In [15]:
df.Normalized_list = df.Normalized_list.apply(literal_eval)
df.Sent_Token = df.Sent_Token.apply(literal_eval)
df.Token = df.Token.apply(literal_eval)

In [16]:
df["Cleaned_text"] = df['Cleaned_text'].values.astype('U')

### Markav Model

In [5]:
train, test = model_selection.train_test_split(df, test_size=0.3)

In [6]:
SpeakerName = df["Speaker"].unique()

In [7]:
SpeakerName

array(['BUSH', 'HILLARY', 'OBAMA', 'SANDERS', 'TRUMP'], dtype=object)

In [8]:
s0 = train.loc[train['Speaker'] == SpeakerName[0]]["RawText"].str.cat(sep=' ')
s1 = train.loc[train['Speaker'] == SpeakerName[1]]["RawText"].str.cat(sep=' ')
s2 = train.loc[train['Speaker'] == SpeakerName[2]]["RawText"].str.cat(sep=' ')
s3 = train.loc[train['Speaker'] == SpeakerName[3]]["RawText"].str.cat(sep=' ')
s4 = train.loc[train['Speaker'] == SpeakerName[4]]["RawText"].str.cat(sep=' ')

In [9]:
sl = [s0,s1,s2,s3,s4]

In [10]:
TOO_FULL = 0.5
GROWTH_RATIO = 2


class Hash_Table:

    def __init__(self, cells, defval):
        '''
        Construct a new hash table with a fixed number of cells equal to the \
        parameter "cells", and which yields the value defval upon a lookup to \
        a key that has not previously been inserted
        '''
        self.occupy = 0
        assert cells > 0 and type(cells)== int, \
            'cells should be an integar larger than 0'
        self.cells = cells
        self.defval = defval
        self.table = [None for x in range(self.cells)]


    def lookup(self, key):
        '''
        Retrieve the value associated with the specified key in the hash \
        table,or return the default value if it has not previously been \
        inserted.
        '''
        place_num = self.hashval(key)
        n = 0
        while self.table[place_num]:
            if self.table[place_num][0] == key:
                return self.table[place_num][1]
            n += 1
            place_num = (place_num + n) % self.cells
        return self.defval

    def update(self, key, val):
        '''
        Change the value associated with key "key" to value "val".\
        If "key" is not currently present in the hash table,  insert it \
        with
        value "val".
        '''
        n = 0
        place_num = self.hashval(key)
        if self.lookup(key) == self.defval:
            while self.table[place_num]:
                n += 1
                place_num = (place_num + n) % self.cells
            self.occupy += 1

        else:
            while self.table[place_num]:
                if self.table[place_num][0] == key:
                    break
                else:
                    n += 1
                    place_num = (place_num + n) % self.cells

        self.table[place_num] = (key, val)

        if self.occupy / self.cells > TOO_FULL:
            self.expand()

    def hashval(self, key, sums=0):
        '''
        Get the original hash value position.
        '''
        if len(key)==0:
            return sums
        sums = (sums*37 + ord(key[0])) % self.cells
        return self.hashval(key[1:], sums)

# Reference: https://stackoverflow.com/questions/16096754/ /
# remove-none-value-from-a-list-without-removing-the-0-value
    def expand(self):
        '''
        Expand the size of the table when the vacant place of the table is \
        less than TOO_FULL*self.cells.
        '''
        valid = list(filter(None.__ne__, self.table))
        self.cells = self.cells * GROWTH_RATIO
        self.table = [None for x in range(self.cells)]
        self.occupy = 0
        for i in valid:
            self.update(i[0], i[1])

In [24]:
import sys
import math

HASH_CELLS = 57


class Markov:

    def __init__(self,k,s):
        '''
        Construct a new k-order Markov model using the statistics of string "s"
        '''
        assert type(s) == str and len(s) > 0, \
            'Please use a valid string.'
        assert type(k) == int and k > 0, \
            'K should be an integar larger than 0.' 
        self.k = k
        self.s = s
        self.lng = len(s)
        self.unique_s = len(set(s))
        self.tb = self.get_mktable()
        
    def log_probability(self, s):
        '''
        Get the log probability of string "s", given the statistics of
        character sequences modeled by this particular Markov model
        This probability is *not* normalized by the length of the string.
        '''
        total = 0
        for i in range(len(s)):
            k = self.start_end(i, self.k, s)
            k_plus = self.start_end(i, self.k+1, s)
            k_count = self.tb.lookup(k)
            k_plus_count = self.tb.lookup(k_plus)
            single_prob = math.log((k_plus_count + 1) / (k_count + \
                self.unique_s))
            total += + single_prob
        return total

    def get_mktable(self):
        '''
        Create a hash table for self.s string. This table considers k\
        characters and k+1 characters.
        '''
        tb = Hash_Table(HASH_CELLS, 0)
        for i in range(self.lng):
            k = self.start_end(i, self.k, self.s)
            k_plus = self.start_end(i, self.k+1, self.s)
            if k:
                k_count = tb.lookup(k)
                tb.update(k, k_count + 1)
            if k_plus:
                k_plus_count = tb.lookup(k_plus)
                tb.update(k_plus, k_plus_count + 1)
        return tb

    def start_end(self, i, k_, s):
        '''
        Given i, k and s, this function will return all sequences.
        E.g. i = 0, k = 2, s = 'abcd', this funciton will return 'ab'.
        '''
        if k_ + i < len(s):
            k = s[i:(i + k_)]
        else:
            k = s[i:] + s[:(i + k_) % len(s)]
        return k

def identify_speaker(Speaker, speech_test, ml):
    '''
    Given sample text from speakers, and text from an unidentified speaker,
    return a tuple with the *normalized* log probabilities of each of the speakers
    uttering that text under a "order" order character-based Markov model,
    and a conclusion of which speaker uttered the unidentified text
    based on the probabilities.
    '''
    model1 = ml[0]
    model2 = ml[1]
    model3 = ml[2]
    model4 = ml[3]
    model5 = ml[4]


    probability1 = model1.log_probability(speech_test) / len(speech_test)
    probability2 = model2.log_probability(speech_test) / len(speech_test)
    probability3 = model3.log_probability(speech_test) / len(speech_test)
    probability4 = model4.log_probability(speech_test) / len(speech_test)
    probability5 = model5.log_probability(speech_test) / len(speech_test)

    
    prolist = [probability1,probability2,probability3,probability4,probability5]


    max_ind = max(enumerate(prolist),key=lambda x: x[1])[0]
    

    return Speaker[max_ind]

In [27]:
def calcaulate_accuracy(test, sl, order, Speaker):
    model1 = Markov(order, sl[0])
    print(1)
    model2 = Markov(order, sl[1])
    print(2)
    model3 = Markov(order, sl[2])
    print(3)
    model4 = Markov(order, sl[3])
    print(4)
    model5 = Markov(order, sl[4])
    print(5)
    ml = [model1,model2,model3,model4,model5]
    
    total_test = test.shape[0]
    right_num = 0
    for ind, row in test.iterrows():
        real_speaker = row['Speaker']
        speech_test = row["RawText"]
        print(real_speaker)
        est_speaker = identify_speaker(sl,order, Speaker, speech_test,ml)
        print(est_speaker)
        if est_speaker == real_speaker:
            right_num += 1 
    return right_num/total_test

In [28]:
k2_rate = calcaulate_accuracy(test, sl, 2,SpeakerName)

1
2
3
4
5
SANDERS
SANDERS
TRUMP
TRUMP
BUSH
BUSH
SANDERS
SANDERS
HILLARY
HILLARY
OBAMA
OBAMA
HILLARY
OBAMA
TRUMP
TRUMP
TRUMP
TRUMP
TRUMP
TRUMP
TRUMP
TRUMP
TRUMP
TRUMP
OBAMA
OBAMA
HILLARY
HILLARY
HILLARY
HILLARY
BUSH
BUSH
TRUMP
TRUMP
TRUMP
TRUMP
HILLARY
HILLARY
BUSH
BUSH
TRUMP
TRUMP
SANDERS
SANDERS
OBAMA
OBAMA
OBAMA
HILLARY
OBAMA
OBAMA
HILLARY
HILLARY
OBAMA
OBAMA
OBAMA
OBAMA
TRUMP
TRUMP
SANDERS
SANDERS
HILLARY
HILLARY
TRUMP
TRUMP
BUSH
BUSH
TRUMP
TRUMP
OBAMA
OBAMA
OBAMA
OBAMA
BUSH
BUSH
HILLARY
HILLARY
TRUMP
TRUMP
HILLARY
HILLARY
OBAMA
OBAMA
SANDERS
SANDERS
SANDERS
SANDERS
HILLARY
HILLARY
OBAMA
OBAMA
OBAMA
OBAMA
SANDERS
HILLARY
OBAMA
OBAMA
BUSH
BUSH
OBAMA
OBAMA
OBAMA
OBAMA
HILLARY
HILLARY
TRUMP
TRUMP
SANDERS
SANDERS
TRUMP
TRUMP
TRUMP
TRUMP
OBAMA
OBAMA
TRUMP
TRUMP
BUSH
BUSH
OBAMA
OBAMA
BUSH
BUSH
BUSH
BUSH
HILLARY
SANDERS
OBAMA
HILLARY
SANDERS
SANDERS
OBAMA
OBAMA
OBAMA
OBAMA
BUSH
BUSH
OBAMA
OBAMA
HILLARY
HILLARY
BUSH
BUSH
OBAMA
OBAMA
TRUMP
TRUMP
BUSH
BUSH
BUSH
BUSH
BUSH
BUSH
HILLARY
OBAMA
SA

In [29]:
k2_rate

0.9407894736842105

In [30]:
k3_rate = calcaulate_accuracy(test, sl, 3,SpeakerName)

1
2
3
4
5
SANDERS
SANDERS
TRUMP
TRUMP
BUSH
BUSH
SANDERS
SANDERS
HILLARY
HILLARY
OBAMA
HILLARY
HILLARY
OBAMA
TRUMP
TRUMP
TRUMP
TRUMP
TRUMP
TRUMP
TRUMP
TRUMP
TRUMP
TRUMP
OBAMA
OBAMA
HILLARY
HILLARY
HILLARY
HILLARY
BUSH
BUSH
TRUMP
TRUMP
TRUMP
TRUMP
HILLARY
HILLARY
BUSH
BUSH
TRUMP
TRUMP
SANDERS
SANDERS
OBAMA
HILLARY
OBAMA
HILLARY
OBAMA
OBAMA
HILLARY
HILLARY
OBAMA
OBAMA
OBAMA
OBAMA
TRUMP
TRUMP
SANDERS
SANDERS
HILLARY
HILLARY
TRUMP
TRUMP
BUSH
BUSH
TRUMP
TRUMP
OBAMA
HILLARY
OBAMA
OBAMA
BUSH
BUSH
HILLARY
HILLARY
TRUMP
TRUMP
HILLARY
HILLARY
OBAMA
HILLARY
SANDERS
SANDERS
SANDERS
SANDERS
HILLARY
HILLARY
OBAMA
OBAMA
OBAMA
HILLARY
SANDERS
HILLARY
OBAMA
OBAMA
BUSH
BUSH
OBAMA
OBAMA
OBAMA
OBAMA
HILLARY
HILLARY
TRUMP
TRUMP
SANDERS
SANDERS
TRUMP
TRUMP
TRUMP
TRUMP
OBAMA
OBAMA
TRUMP
TRUMP
BUSH
BUSH
OBAMA
OBAMA
BUSH
BUSH
BUSH
BUSH
HILLARY
HILLARY
OBAMA
HILLARY
SANDERS
SANDERS
OBAMA
OBAMA
OBAMA
OBAMA
BUSH
BUSH
OBAMA
OBAMA
HILLARY
HILLARY
BUSH
BUSH
OBAMA
OBAMA
TRUMP
TRUMP
BUSH
BUSH
BUSH
BUSH
BUSH
BUSH
HILLAR

In [31]:
k3_rate

0.8881578947368421

### Bag of word model

In [ ]:
# based on words

In [17]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(df['Cleaned_text'], 
                                                    df['Speaker'], 
                                                    test_size=0.1, 
                                                    random_state=42,
                                                    stratify=df['Speaker'])

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(453,) (51,) (453,) (51,)


In [18]:
vectorizer_word = TfidfVectorizer(max_features=40000,
                             min_df=5, 
                             max_df=0.5, 
                             analyzer='word', 
                             stop_words='english', 
                             ngram_range=(1, 2))

vectorizer_word.fit(x_train)

tfidf_matrix_word_train = vectorizer_word.transform(x_train)
tfidf_matrix_word_test = vectorizer_word.transform(x_test)

In [19]:
lr_word = LogisticRegression(solver='sag', verbose=2)
lr_word.fit(tfidf_matrix_word_train, y_train)

convergence after 19 epochs took 1 seconds
convergence after 21 epochs took 0 seconds
convergence after 20 epochs took 0 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


convergence after 22 epochs took 0 seconds
convergence after 23 epochs took 0 seconds


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.4s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=2, warm_start=False)

In [20]:
# joblib.dump(lr_word, './models/lr_word_ngram.pkl')

y_pred_word = lr_word.predict(tfidf_matrix_word_test)
pd.DataFrame(y_pred_word, columns=['y_pred']).to_csv('test12.csv', index=False)

In [33]:
y_pred_word = pd.read_csv('test12.csv')
print("Bag-of-words(words) Accuracy Score -> ",accuracy_score(y_test, y_pred_word)*100)

Bag-of-words(words) Accuracy Score ->  86.27450980392157


In [ ]:
# based on characters

In [22]:
from tqdm import tqdm_notebook

In [23]:
vectorizer_char = TfidfVectorizer(max_features=40000,
                             min_df=5, 
                             max_df=0.5, 
                             analyzer='char', 
                             ngram_range=(1, 4))

vectorizer_char.fit(tqdm_notebook(x_train, leave=False));

tfidf_matrix_char_train = vectorizer_char.transform(x_train)
tfidf_matrix_char_test = vectorizer_char.transform(x_test)

lr_char = LogisticRegression(solver='sag', verbose=2)
lr_char.fit(tfidf_matrix_char_train, y_train)

y_pred_char = lr_char.predict(tfidf_matrix_char_test)
# joblib.dump(lr_char, './models/lr_char_ngram.pkl')

pd.DataFrame(y_pred_char, columns=['y_pred']).to_csv('test13.csv', index=False)

HBox(children=(IntProgress(value=0, max=453), HTML(value='')))

convergence after 30 epochs took 0 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


convergence after 25 epochs took 1 seconds
convergence after 23 epochs took 0 seconds
convergence after 29 epochs took 1 seconds
convergence after 30 epochs took 0 seconds


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.4s finished


In [35]:
y_pred_chary_pred_c  = pd.read_csv('test13.csv')
print("Bag-of-words(character) Accuracy Score -> ",accuracy_score(y_test, y_pred_char)*100)

Bag-of-words(character) Accuracy Score ->  84.31372549019608


In [25]:
# based on words+characters

In [26]:
tfidf_matrix_word_char_train =  hstack((tfidf_matrix_word_train, tfidf_matrix_char_train))
tfidf_matrix_word_char_test =  hstack((tfidf_matrix_word_test, tfidf_matrix_char_test))

lr_word_char = LogisticRegression(solver='sag', verbose=2)
lr_word_char.fit(tfidf_matrix_word_char_train, y_train)

y_pred_word_char = lr_word_char.predict(tfidf_matrix_word_char_test)
# joblib.dump(lr_word_char, './models/lr_word_char_ngram.pkl')

pd.DataFrame(y_pred_word_char, columns=['y_pred']).to_csv('test14.csv', index=False)

convergence after 51 epochs took 1 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


convergence after 45 epochs took 2 seconds
convergence after 42 epochs took 1 seconds
convergence after 53 epochs took 1 seconds
convergence after 56 epochs took 1 seconds


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    5.7s finished


In [36]:
y_pred_word_char = pd.read_csv('test14.csv')
print("Bag-of-words(words+characters) Accuracy Score -> ",accuracy_score(y_test, y_pred_word_char)*100)

Bag-of-words(words+characters) Accuracy Score ->  88.23529411764706


### Naive Bayesian and SVM based on tokens

In [29]:
np.random.seed(500)

In [30]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [31]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['Cleaned_text'],df['Speaker'],test_size=0.3)
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df['Cleaned_text'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  78.28947368421053


In [32]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  92.76315789473685


### KNN

In [38]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(df['Cleaned_text'], 
                                                    df['Speaker'], 
                                                    test_size=0.1, 
                                                    random_state=42,
                                                    stratify=df['Speaker'])

x_train_n = []
for i in x_train:
    x_train_n.append(i)

x_test_n = []
for i in x_test:
    x_test_n.append(i)

y_train_n = y_train.values
y_test_n = y_test.values

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(x_train_n)

modelknn = KNeighborsClassifier(n_neighbors=9)
modelknn.fit(X,y_train_n)

Test = vectorizer.transform(x_test_n) 
predicted_labels_knn = modelknn.predict(Test)

num = 0
for i in range(len(y_test_n)):
    if y_test_n[i] == predicted_labels_knn[i]:
        num += 1
print("KNN Accuracy Score -> ",num/len(y_test_n)*100)

KNN Accuracy Score ->  72.54901960784314


### Maximum Entropy

In [42]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(df["Token"], 
                                                    df['Speaker'], 
                                                    test_size=0.1, 
                                                    random_state=42,
                                                    stratify=df['Speaker'])
train = pd.concat([x_train,y_train], axis=1, join='inner')
test = pd.concat([x_test,y_test], axis=1, join='inner')

train_set_entropy = [(row['Token'],row['Speaker']) for ind, row in train.iterrows()]
def list_to_dict(words_list):
    return dict([(word, True) for word in words_list])
 
training_set_formatted = [(list_to_dict(element[0]), element[1]) for element in train_set_entropy]

numIterations = 100
 
algorithm = nltk.classify.MaxentClassifier.ALGORITHMS[0]
classifier = nltk.MaxentClassifier.train(training_set_formatted, algorithm, max_iter=numIterations)
classifier.show_most_informative_features(10)

  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.60944        0.170
             2          -1.59792        0.841
             3          -1.58652        0.841
             4          -1.57525        0.848
             5          -1.56409        0.852
             6          -1.55305        0.857
             7          -1.54213        0.863
             8          -1.53133        0.870
             9          -1.52064        0.876
            10          -1.51007        0.876
            11          -1.49961        0.881
            12          -1.48927        0.883
            13          -1.47904        0.883
            14          -1.46892        0.885
            15          -1.45891        0.890
            16          -1.44902        0.892
            17          -1.43923        0.896
            18          -1.42955        0.898
            19          -1.41997        0.903
 

In [43]:
test_set_entropy = [(row['Token'],row['Speaker']) for ind, row in test.iterrows()]
test_set_formatted = [(list_to_dict(element[0]), element[1]) for element in test_set_entropy]

for review in test_set_formatted:
    label = review[1]
    text = review[0]
    determined_label = classifier.classify(text)
    num = 0
    if determined_label==label:
        num+=1
print("The Maximum Entropy Accuracy Score -> ", num/len(test_set_formatted[0])*100)

The Maximum Entropy Accuracy Score ->  50.0
